In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [3]:
df = pd.read_csv('Bhagvad_Gita.csv')

# Step 2: Extract columns into lists
explanations = df['Explanation'].tolist()
shlokas = df['Shloka'].tolist()
chapter_numbers = df['Chapter No'].tolist()
verse_numbers = df['Verse No'].tolist()

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 4: Create embeddings for the Explanation column
explanation_embeddings = model.encode(explanations, show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [8]:
from transformers import pipeline
summarizer = pipeline('summarization', model='t5-small')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [13]:
def retrieve_verse(query, top_k=1):
    # Encode the query
    query_embedding = model.encode(query)

    # Compute cosine similarities
    similarities = util.cos_sim(query_embedding, explanation_embeddings)

    # Find the top-k most similar explanations
    top_results = similarities.topk(top_k)

    # Return the Shloka, Chapter No, and Verse No with the matching Explanation
    results = []
    for idx in top_results.indices[0]:
        explanation = explanations[idx]
        summarized_explanation = summarizer(explanation, max_length=100, min_length=20, do_sample=False)[0]['summary_text']
        results.append({
            "Chapter No": chapter_numbers[idx],
            "Verse No": verse_numbers[idx],
            "Shloka": shlokas[idx],
            "Summary": summarized_explanation,
            "Similarity Score": similarities[0][idx].item()
        })
    return results

In [15]:
generator = pipeline('text-generation', model='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [32]:
def retrieve_verse(query, top_k=1):
    # Encode the query
    query_embedding = model.encode(query)

    # Compute cosine similarities
    similarities = util.cos_sim(query_embedding, explanation_embeddings)

    # Find the top-k most similar explanations
    top_results = similarities.topk(top_k)

    # Generate human-like responses
    results = []
    for idx in top_results.indices[0]:
        explanation = explanations[idx]
        prompt = f"Based on this teaching: {explanation}Provide practical advice for someone dealing with common life problems, in simple and relatable language."
        response = generator(prompt, max_new_tokens=100, do_sample=True)[0]['generated_text']
        summarized_explanation = summarizer(response, max_length=100, min_length=20, do_sample=False)[0]['summary_text']


        results.append({
            "Chapter No": chapter_numbers[idx],
            "Verse No": verse_numbers[idx],
            "Shloka": shlokas[idx],
            "Generated Response": summarized_explanation,
            "Similarity Score": similarities[0][idx].item()
        })
    return results

In [34]:
query = "How can I stay calm during difficult times?"
result = retrieve_verse(query)

# Display the result
for res in result:
    print(f"Chapter No: {res['Chapter No']}, Verse No: {res['Verse No']}")
    print(f"Shloka: {res['Shloka']}")
    print(f"Generated Response: {res['Generated Response']}")
    print(f"Similarity Score: {res['Similarity Score']:.4f}")
    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chapter No: 4, Verse No: 10
Shloka: वीतरागभयक्रोधा मन्मया मामुपाश्रिता: | बहवो ज्ञानतपसा पूता मद्भावमागता: ॥10॥
Generated Response: in the previous verse, a question may arise whether Lord Krishna is partial in bestowing his grace upon those who absorb their minds in him versus the worldly-minded souls . the Supreme Lord addresses this in the next verse .
Similarity Score: 0.3364
--------------------------------------------------
